In [ ]:
import wfcommons.wfchef.utils as utils

from QHyper.problems.algorithms.utils import wfworkflow_to_qhyper_workflow, merge_subworkflows
from QHyper.problems.algorithms.workflow_decomposition import HeftBasedAlgorithm
from QHyper.problems.workflow_scheduling import Workflow, WorkflowSchedulingProblem
from QHyper.solvers.cqm import CQM

## Load and divide workflow

In [ ]:
tasks_file = "workflows_data/workflows/complex_workflow.json"
machines_file = "workflows_data/machines/machines_for_3_tasks_3_machines_1_path.json"
deadline = 50

In [ ]:
workflow = Workflow(tasks_file, machines_file, deadline)
utils.draw(workflow.wf_instance.workflow, with_labels=True)

In [ ]:
wf_workflow1, wf_workflow2 = HeftBasedAlgorithm().decompose(workflow, 2)
merged = merge_subworkflows([wf_workflow1, wf_workflow2])
utils.draw(merged, with_labels=True)

In [ ]:
workflow1 = wfworkflow_to_qhyper_workflow(wf_workflow1, machines_file, deadline / 2)
workflow2 = wfworkflow_to_qhyper_workflow(wf_workflow2, machines_file, deadline / 2)
wsp1 = WorkflowSchedulingProblem(workflow1)
wsp2 = WorkflowSchedulingProblem(workflow2)

## Solve the Workflow Schedluling Problem with the CQM solver.

In [ ]:
cqm1 = CQM(problem=wsp1, time=5)
cqm2 = CQM(problem=wsp2, time=5)

solution_cqm1 = cqm1.solve()
solution_cqm2 = cqm2.solve()